In [25]:
import cv2
import sys
import os
import numpy as np

In [26]:
#Extraer datos de una ruta
from pathlib import Path

def ls3(path):
    return [obj.name for obj in Path(path).iterdir() if obj.is_file()]

In [27]:
#Contar el numero de fotos en la ruta 'path'
path = r'C:\Users\caroj\Documents\Maestria_AI\ProcesamientoImagen\otras'
a = ls3(path)
image = []
total =0
for image_name in a:
    path_file = os.path.join(path, image_name)
    imagen = cv2.imread(path_file)
    image.append(imagen)
    total += 1
# Imprime el total de imagenes a trabajar
print("El total de imagenes para el ejercicio es de", total )
# Solicita al usuario el numero de imagen de referencia
ref = int(input("Ingrese el numero de imagen de referencia: "))


El total de imagenes para el ejercicio es de 3


Ingrese el numero de imagen de referencia:  2


In [28]:
def click(event, x, y, flags, param): # permite enconcontarr los puntos clickeados 
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))

In [29]:
# Metodo que permite generar la homografia apartir de un par de imagenes concatenadas; tiene dos parametros 1. la imagen concatenada 2. un arreglo de puntos inicializados en vacio retorna la matriz de homografia
def generarHomografia(concatenada, points) :
    width = image[0].shape[1]
    image_draw = np.copy(concatenada)
    #image_draw = cv2.hconcat(np.copy(image[0]),np.copy(image[1]))
    pointsN = []
    points1 = []
    points2 = []
    
    cv2.namedWindow("Image")
    cv2.setMouseCallback("Image", click)

    point_counter = 0
    
    while True:
        
        cv2.imshow("Image", image_draw)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("x"):
            pointsN = points.copy()
            points = []
            break
        if len(points) > point_counter:
            point_counter = len(points)
             #Dado que los puntos son intercalados entonces se asume que los pares son azules y los impares son rojos
            if point_counter % 2 == 0:
                cv2.circle(image_draw, (points[-1][0], points[-1][1]), 3, [0, 0, 255], -1)
            else:
                cv2.circle(image_draw, (points[-1][0], points[-1][1]), 3, [255, 0, 0], -1)
    count = 0
    # Se dividen en dos arreglos los puntos azules points1 y los rojos points2
    for i in pointsN: # extraer elementos de posiciones imapres
        if count % 2 == 0:
            points1.append(i)
        else:
            points2.append(i)
        count += 1
        
   # Teniendo en cuenta que la posicion de los puntos se ve afectada por lo cual se debe hacer una correccion de posicion en los points2
    puntos2 =[]
    for k in points2:
        puntos2.append((k[0]- width,k[1]) )

    # Revision de 4 puntos
    N = min(len(points1), len(points2))
    assert N >= 4, 'At least four points are required'
    
    pts1 = np.array(points1[:N])
    pts2 = np.array(puntos2[:N])
    if False:
        H, _ = cv2.findHomography(pts1, pts2, method=0)
    else:
        H, _ = cv2.findHomography(pts1, pts2, method=cv2.RANSAC)
        
    cv2.waitKey(0)
    cv2.destroyWindow("Image")
    #retorna h -> la homografia del par de fotos
    return H
    
    

In [30]:
# ---------- GENERACION DE TODOS LOS PARES DE FOTOGRAFIAS Y PROCESAMIENTO PARA OBTENER LAS HOMGRAFIAS POR PARES -----------------------------------------------------
# Construir imagen concatenada y alamcenamiento 
Concatenadas = []
for i in range(1,total):
    x = np.concatenate((image[i-1], image[i]), axis=1)  # se concatenan las 2 imagenes 
    Concatenadas.append(x) # Agregar la imagen concatenada x en el arreglo Concatenadas 
    
# Procesamiento 
Homografias = [] # En este arreglo se almacenan las homografias resultado de la concatenacion
for h in Concatenadas:
     
    points = [] # Puntos iniciales parametro del metodo generarHomografia
    a = generarHomografia(h,points) # en a se alamcena la homografia resultalte de la imagen Image[i-1] , image[i]
    Homografias.append(a)
    


In [32]:
# Encontrar las homografias basados en la imagen referencia indicada por el usuario
ref = ref-1
resultados = [] # Alamcena las proyecciones de las homgrafias en relacion a la imagen de referencia
for i in range(0, len(Homografias)):
    # genera las tranformaciones de la primera mitad de utilizando la imagen de referencia como la mitad 
    if (ref) > i:
        
        if i - (ref-1) ==0:
            resultados.append(x[i]) 
        else:
            contador= i
            mult =x[contador]
            while contador!= ref-1:
                mult = mult * x[contador+1]
                contador += 1
            resultados.append(mult)
    # genera las tranformaciones de la segunda mitad de utilizando la imagen de referencia como la mitad 
    else:
        if i - (ref) ==0:
            resultados.append(x[i])
        else:
            contador= i
            mult =x[contador]
            while contador!= ref:
                mult = mult * x[contador-1]
                contador -= 1
            a = np.linalg.inv(mult)
            resultados.append(a)    

In [31]:

print(Homografias[1])

[[ 9.40404902e-01  7.71293203e-03 -1.86987540e+02]
 [ 8.43478730e-03  9.91783212e-01 -4.91132692e+00]
 [ 3.19569434e-05 -3.10633748e-05  1.00000000e+00]]


In [33]:
transformadas = []


image_warped1 = cv2.warpPerspective(image[0], Homografias[0], (image[0].shape[1], image[0].shape[0]))
image_warped2 = cv2.warpPerspective(image[2], Homografias[1], (image[2].shape[1], image[2].shape[0]))

suma = cv2.add(image_warped1, image[1], image_warped2)
cv2.imshow("Suma", suma)
cv2.waitKey(0)
cv2.destroyWindow("Suma")